In [1]:
# https://chromedriver.chromium.org/downloads

In [2]:
# For Image Scrapping 'selenium' library is used here
from selenium import webdriver
DRIVER_PATH = "C:/Users/Dell/Downloads/chromedriver_win32/chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\Dell\AppData\Local\Temp\ipykernel_2556\3096783753.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
# Importing necessary libraries
import os
import time
import io
from PIL import Image
import hashlib
import requests
from bs4 import BeautifulSoup

In [4]:
# Defining a function to fetch the image urls so that images could be downloaded

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [19]:
# Defining a function to open the images in a new window of chrome browser
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=100)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [20]:
# Defining a function to search images according to search term(given by user) and download it into
# our local computer

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [21]:
search_term='APPLE wallpapers'
search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

C:\Users\Dell\AppData\Local\Temp\ipykernel_2556\1194074995.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\Dell\AppData\Local\Temp\ipykernel_2556\4128504895.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\Dell\AppData\Local\Temp\ipykernel_2556\4128504895.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 5 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTtzQSEhGIy6C6nb8e4mslhsG4rpf5tDNiGqA&usqp=CAU - as ./images\apple_wallpapers\1b1c9bf299.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnaBbOYIfH5VhypjpY9cIt2Xrm4DBr7A4cbA&usqp=CAU - as ./images\apple_wallpapers\400ac11a69.jpg
SUCCESS - saved https://i.pinimg.com/736x/b6/96/c0/b696c0a598a2463254cd701a6b6eae2e.jpg - as ./images\apple_wallpapers\5041c43a12.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXWts2uBrhiEsbnRM889SipEScwLgGF0QUCw&usqp=CAU - as ./images\apple_wallpapers\41825b8063.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSLdnw8v1FQtig3nxBaqutJXK1wkUbnZzs1w&usqp=CAU - as ./images\apple_wallpapers\da608b84b6.jpg


In [13]:
# The images are downoloaded into following path of my computer

# C:\Users\Dell\Sciffer Analytics Internship\images